# **目標**:
### 使用SQuAD 2.0 資料集和facebook/bart-base 訓練 Question-Answering 模型生成 Answer

# 安裝套件

In [ ]:
!pip install transformers datasets accelerate

# 確認 GPU 分配

In [ ]:
!nvidia-smi

Sat Apr  9 15:28:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 連雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 資料下載

In [ ]:
%cd /content/drive/Shareddrives/中興大學/讀書會/DataSet/SQuAD

/content/drive/Shareddrives/中興大學/讀書會/DataSet/SQuAD


In [ ]:
%ls

dev-v2.0.json  model/  train-v2.0.json


### SQuAD 資料格式
Stanford 大學所整理的閱讀理解資料集 Stanford Question Answering Dataset (SQuAD) 
內容從維基百科中收集超過 10 萬筆的 CQA pair


For more information please refer to Paper: https://arxiv.org/abs/1606.05250

### Data format 資料格式

- version : <String> 資料集版本
- data : <Array>
  - title : <String> : 文章標題
  - id : <String> : 文章編號
  - paragraphs : <Array>
    - id : <String> : 文章編號_段落編號
    - context : <String> : 段落內容
    - qas : <Array>
      - question : <String> : 問題內容
      - id :<String> : 文章編號_段落編號_問題編號
      - is_impossible : <String> : "1"表示為不可回答，"2"為可回答
      - answers : <Arrays>
        - answer_start : <int> text在文中位置
        - text : <string> : 答案內容


In [ ]:
import json
from pprint import pprint
with open('dev-v2.0.json') as file:
  train_data = json.load(file)

for ele in train_data['data']:
  pprint(ele['paragraphs'][0])
  break

{'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: '
            'Normanni) were the people who in the 10th and 11th centuries gave '
            'their name to Normandy, a region in France. They were descended '
            'from Norse ("Norman" comes from "Norseman") raiders and pirates '
            'from Denmark, Iceland and Norway who, under their leader Rollo, '
            'agreed to swear fealty to King Charles III of West Francia. '
            'Through generations of assimilation and mixing with the native '
            'Frankish and Roman-Gaulish populations, their descendants would '
            'gradually merge with the Carolingian-based cultures of West '
            'Francia. The distinct cultural and ethnic identity of the Normans '
            'emerged initially in the first half of the 10th century, and it '
            'continued to evolve over the succeeding centuries.',
 'qas': [{'answers': [{'answer_start': 159, 'text': 'France'},
              

# 分析模型 (計算 exact match, F1-score )

In [ ]:
from transformers import BartTokenizerFast, BartForConditionalGeneration, AutoConfig, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm
import json

In [ ]:
%cd /content/drive/Shareddrives/中興大學/讀書會/DataSet/SQuAD/
%ls

/content/drive/Shareddrives/中興大學/讀書會/DataSet/SQuAD
dev-v2.0.json  model/  train-v2.0.json


# 載入模型與測試資料

In [ ]:
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")
config = AutoConfig.from_pretrained("./model/CG_epoch_1/config.json") 
model = BartForConditionalGeneration.from_pretrained("./model/CG_epoch_1/pytorch_model.bin", config=config)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

## 查看模型架構

In [ ]:
print(model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

#讀取資料

In [ ]:
from pathlib import Path
def read_data(path, limit=None):
    path = Path(path)
    with open(path, 'rb') as f:
        data_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    unanswers = {'text': '', 'answer_start': 0}
    for group in data_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                answer_list = qa['answers']
                if len(answer_list) == 0:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(unanswers)
                else:
                  for answer in answer_list:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
                  if limit != None and len(contexts) > limit:
                    return contexts, questions, answers
                  
    return contexts, questions, answers

In [ ]:
eval_contexts, eval_questions, eval_answers = read_data('dev-v2.0.json',100)

In [ ]:
print("1st eval context = ",eval_contexts[0])
print("1st eval question = ",eval_questions[0])
print("1st eval answers = ",eval_answers[0])

1st eval context =  The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
1st eval question =  In what country is Normandy located?
1st eval answers =  {'text': 'France', 'answer_start': 159}


#將資料進行Tokenize

In [ ]:
eval_encodings = tokenizer(eval_contexts, eval_questions, truncation=True, padding=True)

In [ ]:
eval_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
eval_encodings[0]

Encoding(num_tokens=294, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

###新增Label

In [ ]:
def add_label(encoding,answers):
  labels = []
  for answer in answers:
    label_tokens = tokenizer.tokenize( answer['text'], max_length=config.max_length, padding="max_length", truncation=True)
    lable_ids = tokenizer.convert_tokens_to_ids(label_tokens)
    labels.append(lable_ids)
    
    encoding.update({'labels': labels})

In [ ]:
def add_label(encoding,answers):
  labels = []
  for answer in answers:
    label_tokens = tokenizer.tokenize( answer['text'] + tokenizer.eos_token, max_length=config.max_length, padding="max_length", truncation=True)
    lable_ids = tokenizer.convert_tokens_to_ids(label_tokens)
    labels.append(lable_ids)
    
    encoding.update({'labels': labels})

In [ ]:
def add_label(encoding,answers):
  labels = []
  for answer in answers:
    label_tokens = tokenizer.tokenize( tokenizer.bos_token + answer['text'] + tokenizer.eos_token, max_length=config.max_length, padding="max_length", truncation=True)
    lable_ids = tokenizer.convert_tokens_to_ids(label_tokens)
    labels.append(lable_ids)
    
    encoding.update({'labels': labels})

In [ ]:
add_label(eval_encodings, eval_answers)

In [ ]:
eval_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
print(eval_answers[0]['text'])

France


In [ ]:
print("input_ids\n", eval_encodings['input_ids'][0])
print("input_ids to tokens\n",tokenizer.convert_ids_to_tokens(eval_encodings['input_ids'][0]))
print("input_ids_decode\n", tokenizer.decode(eval_encodings['input_ids'][0]))
print("attention_mask\n", eval_encodings['attention_mask'][0])
print("labels\n", eval_encodings['labels'][0])
print("labels\n", tokenizer.decode(eval_encodings['labels'][0]))

input_ids
 [0, 133, 20336, 1253, 36, 487, 16803, 35, 234, 2126, 119, 8771, 131, 1515, 35, 20336, 8771, 131, 5862, 35, 20336, 28867, 43, 58, 5, 82, 54, 11, 5, 158, 212, 8, 365, 212, 11505, 851, 49, 766, 7, 37741, 6, 10, 976, 11, 1470, 4, 252, 58, 22306, 31, 41498, 6697, 487, 16803, 113, 606, 31, 22, 487, 27209, 397, 8070, 10369, 268, 8, 34941, 31, 10060, 6, 14605, 8, 8683, 54, 6, 223, 49, 884, 13065, 139, 6, 1507, 7, 24909, 10668, 12107, 7, 1745, 3163, 6395, 9, 580, 17932, 493, 4, 6278, 6808, 9, 8446, 43616, 8, 17793, 19, 5, 3763, 3848, 1173, 8, 7733, 12, 534, 6695, 1173, 9883, 6, 49, 29285, 74, 9097, 19388, 19, 5, 9347, 154, 811, 12, 805, 13426, 9, 580, 17932, 493, 4, 20, 11693, 4106, 8, 7289, 3599, 9, 5, 20336, 1253, 4373, 3225, 11, 5, 78, 457, 9, 5, 158, 212, 3220, 6, 8, 24, 1143, 7, 14842, 81, 5, 27544, 11505, 4, 2, 2, 1121, 99, 247, 16, 37741, 2034, 116, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
print("input_ids\n", eval_encodings['input_ids'][21])
print("input_ids to tokens\n",tokenizer.convert_ids_to_tokens(eval_encodings['input_ids'][21]))
print("input_ids_decode\n", tokenizer.decode(eval_encodings['input_ids'][21]))
print("attention_mask\n", eval_encodings['attention_mask'][21])
print("labels\n", eval_encodings['labels'][21])
print("labels\n", tokenizer.decode(eval_encodings['labels'][21]))

input_ids
 [0, 133, 20336, 1253, 36, 487, 16803, 35, 234, 2126, 119, 8771, 131, 1515, 35, 20336, 8771, 131, 5862, 35, 20336, 28867, 43, 58, 5, 82, 54, 11, 5, 158, 212, 8, 365, 212, 11505, 851, 49, 766, 7, 37741, 6, 10, 976, 11, 1470, 4, 252, 58, 22306, 31, 41498, 6697, 487, 16803, 113, 606, 31, 22, 487, 27209, 397, 8070, 10369, 268, 8, 34941, 31, 10060, 6, 14605, 8, 8683, 54, 6, 223, 49, 884, 13065, 139, 6, 1507, 7, 24909, 10668, 12107, 7, 1745, 3163, 6395, 9, 580, 17932, 493, 4, 6278, 6808, 9, 8446, 43616, 8, 17793, 19, 5, 3763, 3848, 1173, 8, 7733, 12, 534, 6695, 1173, 9883, 6, 49, 29285, 74, 9097, 19388, 19, 5, 9347, 154, 811, 12, 805, 13426, 9, 580, 17932, 493, 4, 20, 11693, 4106, 8, 7289, 3599, 9, 5, 20336, 1253, 4373, 3225, 11, 5, 78, 457, 9, 5, 158, 212, 3220, 6, 8, 24, 1143, 7, 14842, 81, 5, 27544, 11505, 4, 2, 2, 2264, 16, 1470, 10, 976, 9, 116, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

# 定義 Dataset，並轉換成 tensor 格式

In [ ]:
import torch
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
eval_dataset = Dataset(eval_encodings)

In [ ]:
eval_batch_size = 2     # 設定 batch size
data_collator = default_data_collator

eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, eval_dataloader = accelerator.prepare(
    model, eval_dataloader
)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Evaluation

In [ ]:
print("***** Running eval *****")
model.eval()
ref = []
pre = []
index = 0
true_answer = eval_answers

for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):

  outputs = model(**batch)
  loss = outputs.loss

  
  # for i in range(len(pred_answer)):
  #   score = confidence[i]
  #   text = pred_answer[i]
  #   pre.append({'prediction_text': text,'confidence':score ,'id': str(index)})
  #   index+=1


***** Running eval *****


Eval Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

In [ ]:
ref = []
for i in range(len(eval_answers)):
  start = eval_answers[i]['answer_start']
  text = eval_answers[i]['text']
  ref.append({'answers':{'answer_start':[start],'text':[text]},'id':str(i)})

In [ ]:
for data in pre:
  data['no_answer_probability'] = data['confidence']
  data.pop('confidence',None)

In [ ]:
origin = pre

In [ ]:
for data in pre:
  text = data['prediction_text']
  if len(text) > 1 and text[0] == ' ':
    text = text[1:]
    data['prediction_text'] = text

In [ ]:
print(ref[0])
print(pre[0])

{'answers': {'answer_start': [159], 'text': 'France'}, 'id': '0'}
{'prediction_text': 'France', 'id': '0', 'no_answer_probability': 0.8058395087718964}


In [ ]:
for i in range(87,100):
  print(i)
  print('Context:',eval_contexts[i])
  print('Question:',eval_questions[i])
  print('Answer:',ref[i]['answers']['text'][0])
  print('Prediction:',pre[i]['prediction_text'])
  print('-------------\n')

#Inference

In [ ]:
# **撰寫預測程式**
def QA_model(model, context, question):
  input_encodings = tokenizer([context], [question], truncation=True, padding=True)
  input_dataset = Dataset(input_encodings)
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)
  for batch in input_dataloader:
    output_ids = model.generate(**batch, num_beams=3, max_length=20, early_stopping=True)
    predictions_text = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in output_ids]


  for i in range(len(predictions_text)):
    if len(predictions_text[i]) > 1 and predictions_text[i][0] == ' ':
      predictions_text[i] = predictions_text[i][1:]

  return predictions_text


In [ ]:
test_context = 'Hank ate pizza at 6 p.m yesterday.'
test_question = 'When did Hank eat pizza yesterday?'

In [ ]:
test_context = '''Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people).'''
test_question = 'Who is the author of harry potter?'

In [ ]:
test_context =  "One of the first Norman mercenaries to serve as a Byzantine general was Hervé in the 1050s. By then however, there were already Norman mercenaries serving as far away as Trebizond and Georgia. They were based at Malatya and Edessa, under the Byzantine duke of Antioch, Isaac Komnenos. In the 1060s, Robert Crispin led the Normans of Edessa against the Turks. Roussel de Bailleul even tried to carve out an independent state in Asia Minor with support from the local population, but he was stopped by the Byzantine general Alexius Komnenos."
test_question = "When did Herve go up against the Turks?"

In [ ]:
test_context =  "One of the first Norman mercenaries to serve as a Byzantine general was Hervé in the 1050s. By then however, there were already Norman mercenaries serving as far away as Trebizond and Georgia. They were based at Malatya and Edessa, under the Byzantine duke of Antioch, Isaac Komnenos. In the 1060s, Robert Crispin led the Normans of Edessa against the Turks. Roussel de Bailleul even tried to carve out an independent state in Asia Minor with support from the local population, but he was stopped by the Byzantine general Alexius Komnenos."
test_question = "When did Herve serve as a Byzantine general?"

In [ ]:
answer = QA_model(model,test_context,test_question)
print(answer)

['p.m']
